<a href="https://colab.research.google.com/github/44244432664/AIproj/blob/main/svm_intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install underthesea
!pip install underthesea[deep]
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2

In [45]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from underthesea import word_tokenize
from underthesea import text_normalize
from underthesea import dependency_parse
import csv
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from textattack.augmentation import EasyDataAugmenter

import random
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()
# nltk.download('stopwords')

# from nltk.stem import PorterStemmer
# ps = PorterStemmer()


In [5]:
pd.read_csv('QAI.csv')

,Q,A,I
0,Các Trường đại học khác có ngành Công nghệ thô...,"\nTại ĐH Bách Khoa, Khoa KH & KT Máy tính, đào...",4
1,Khác nhau giữa ngành KHMT và KTMT?,\nKHMT:\nNgành Khoa học Máy tính thuộc nhóm ng...,4
2,"Nếu ban đầu em học ngành KHMT, sau đó, em thấy...",\nSV được phép chuyển ngành nếu thỏa các điều ...,5
3,"Em muốn học làm robot, điều khiển từ xa các th...",\nKỹ thuật máy tính là 1 lựa chọn phù hợp.\n,6
4,"Học phí của ngành KHMT, KTMT?",\nTham khảo quy định về mức thu học phí năm họ...,7
...,...,...,...
57,Cho em hỏi phương thức kết hợp khi nào mới có ...,"\nChào bạn, hiện tại vẫn chưa có thông tin về ...",19
58,Dạ em là thí sinh chuẩn bị thi đại học năm nay...,"\nChào bạn, về vấn đề mật khẩu của tài khoản m...",21
59,Dạ cho em hỏi trọng số các tiêu chí điểm trong...,\nNăm 2023 hiện đang để trọng số của các cột đ...,1
60,NaN,\nBộ hồ sơ Ưu tiên xét tuyển theo quy định của...,3


In [6]:
# def read_csv(filename):
#     q = []
#     a = []
#     i = []

#     with open (filename) as csvDataFile:
#         csvReader = csv.reader(csvDataFile)

#         for row in csvReader:
#             q.append(row[0])
#             a.append(row[1])
#             i.append(row[2])

#     Q = np.asarray(q)
#     A = np.asarray(a)
#     # X = phrase
#     I = np.asarray(i,)

#     print(f'Q: {type(Q)}')
#     print(f'A: {type(A)}')
#     print(f'I: {type(I)}')

#     return Q, A, I

stop = pd.read_csv('stopwords.csv')
stop_words = list(stop['stopwords'])
print(stop_words)
# print(Y)

# def stat(Y):
#   a=0
#   b=0
#   c=0
#   for e in Y:
#     if e==0:
#       a+=1
#     if e==1:
#       b+=1
#     if e==2:
#       c+=1
#   return a, b, c

# a, b, c = stat(Y)
# print(a, b, c)

qai = pd.read_csv('QAI.csv')
print(type(qai))
print(type(qai['Q']))
# print(type(qai.frame))
Q = np.asarray(qai['Q'])
A = np.asarray(qai['A'])
I = np.asarray(qai['I'])
Q_I = np.zeros(len(I))
A_I = np.zeros(len(I))
print(f'Q: {Q}, {len(Q)}')
print(f'A: {A}, {len(A)}')
print(f'I: {I}, {len(I)}')

['mà_thôi', 'tuy_có', 'đâu_như', 'quay_bước', 'hỏi_lại', 'khẳng_định', 'giờ_này', 'ren_rén', 'một_cơn', 'ủa', 'chợt', 'tốt_bộ', 'tuổi_tôi', 'mới_hay', 'hơn', 'để_lòng', 'vừa_qua', 'thêm_vào', 'ào_vào', 'nghe_không', 'cái_đã', 'bây_nhiêu', 'cha_chả', 'tự_ý', 'cái_đó', 'không_tính', 'chưa_kể', 'phía', 'tính_từ', 'như_ai', 'anh_ấy', 'lâu_lâu', 'nhất_loạt', 'khó_khăn', 'vùng', 'dù_rằng', 'sự_việc', 'gây_cho', 'phải_như', 'mạnh', 'bước', 'dễ_sử_dụng', 'nhanh_lên', 'bất_quá', 'bởi_vì', 'cả_ngày', 'ô_hay', 'cho_đến', 'ba_ba', 'dưới', 'dạ_bán', 'gặp', 'tránh_xa', 'thật_là', 'vài_điều', 'một_cách', 'ba_cùng', 'vượt_quá', 'vô_kể', 'nghe_được', 'và', 'ô_kê', 'vì', 'nhận_nhau', 'tên', 'riu_ríu', 'không_kể', 'lời_nói', 'trước_đây', 'phải_chi', 'gần_bên', 'chỉ_chính', 'cả_tin', 'nhất_mực', 'cơ_chỉ', 'cao_thế', 'xuống', 'quan_tâm', 'ông_từ', 'mỗi_lần', 'xử_lý', 'là_ít', 'gì_gì', 'quả', 'chú_khách', 'quay_số', 'qua_đi', 'thực_vậy', 'hơn_là', 'làm_mất', 'bỏ_quá', 'nặng', 'dạ_con', 'gặp_khó_khăn', 'các_

In [7]:
# text.lower() to change string to lowercase

In [8]:
# def tokenize(string):
#     return string.split(' ')

In [9]:
def unique_word(set_corpus):
    return (lambda prv, cur: set(prv).union(set(cur)), set_corpus, [])

In [49]:
# X_train, Y_train = read_csv('train.csv')
# # print(X_train.shape)
# X_test, Y_test = read_csv('test.csv')
# # nltk.download('stopwords')
# # stop_words = set(stopwords.words('english'))
# # maxLen = len(max(X_train, key=len).split())
# # index = 1
# # for i in range(len(X_train)):
# #   #  X_train[i] = X_train[i].lower()
# #   # print(X_train[i])
# #   word_tokens = tokenize(X_train[i].lower())
# #   no_sw = [w for w in word_tokens if not w in stop_words]
# #   stem = [ps.stem(w) for w in no_sw]
# #   filtered_sentence = (" ").join(stem)
# #   X_train[i] = filtered_sentence
# #   # print(filtered_sentence)

# # for i in range(len(X_test)):
# #   #  X_test[i] = X_test[i].lower()
# #   # print(X_test[i])
# #   word_tokens = tokenize(X_test[i].lower())
# #   no_sw = [w for w in word_tokens if not w in stop_words]
# #   stem = [ps.stem(w) for w in no_sw]
# #   filtered_sentence = (" ").join(stem)
# #   X_test[i] = filtered_sentence
# #   # print(filtered_sentence)

# # print(X_train[index], '/', Y_train[index])
# # print(X_test[index], '/', Y_test[index])
# # # print(maxLen)
# a_train, b_train, c_train = stat(Y_train)
# a_test, b_test, c_test = stat(Y_test)

# print(f'train: {a_train} 0 label, {b_train} 1 label, {c_train} 2 label')
# print(f'test: {a_test} 0 label, {b_test} 1 label, {c_test} 2 label')
# print(X_train)

# for i in range(len(I)):
#   print(type(Q[i]))
#   print(type(A[i]))
#   print()

# for i in range(len(I)):
#   if type(Q[i]) is not str:
#     Q[i] = ""
#     Q_I[i] = -1
#   else:
#     Q[i] = text_normalize(Q[i])
#     Q[i] = Q[i].lower()
#     Q[i] = word_tokenize(Q[i], format='text')
#     Q_I[i] = I[i]
#     for w in stop_words:
#       Q[i].replace(w, "")

#   if type(A[i]) is not str:
#     A[i] = ""
#     A_I[i] = -1
#   else:
#     A[i] = text_normalize(A[i])
#     A[i] = A[i].lower()
#     A[i] = word_tokenize(A[i], format='text')
#     A_I[i] = I[i]
#     for w in stop_words:
#       A[i].replace(w, "")

# Q = [q for q in Q if q != ""]
# A = [a for a in A if a != ""]
# Q_I = [i for i in Q_I if i != -1]
# A_I = [i for i in A_I if i != -1]

eda_aug = EasyDataAugmenter()

X = []
Y = []

# q = []
# a = []
# intent = []

x = []

for i in range(len(I)):
  if type(Q[i]) is str:
    norm = text_normalize(Q[i])
    low = norm.lower()
    text = word_tokenize(low, format='text')

    aug = (eda_aug.augment(text))

    x = [text] + aug

    for j in range(len(x)):
      for w in stop_words:
        x[j].replace(w, "")
      X.append(x[j])
      Y.append(I[i])

  #     q.append(x[j])
  # else:
  #   q.append("")

  # if type(A[i]) is str:
  #   norm = text_normalize(A[i])
  #   low = norm.lower()
  #   text = word_tokenize(low, format='text')

  #   aug = (eda_aug.augment(text))

  #   x = [text] + aug
  #   for j in range(len(x)):
  #     for w in stop_words:
  #       x[j].replace(w, "")
  #     X_train.append(x[j])
  #     Y_train.append(I[i])

  #     a.append(x[j])
  #     intent.append(I[i])
  # else:
  #   a.append("")
  #   intent.append(I[i])





s = 'đây là Hà Nội, thủ đô của Việt Nam'
ts = word_tokenize(s, format='text')
print(ts)

# print(f'Q {type(Q)}')
# print(f'A {(A)}')
# print(f'I {len(I)}')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


đây là Hà_Nội , thủ_đô của Việt_Nam


In [51]:
with open('new_QAI.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Q", "A", "I"])

    for i in range(len(Y)):
        writer.writerow([X[i], "", Y[i]])

file.close()

In [53]:
pd.read_csv('new_QAI.csv')
rp = np.zeros(22)
t_idx = []
tr_idx = []
idx = []

for i in range(21):
  idx.append([])

for i in range(len(Y)):
  rp[Y[i]-1] += 1
  idx[Y[i]-1].append(i)

print(idx)

for i in range(len(rp)):
  print(f"{i+1}: {rp[i]}")

print(sum(rp))

for l in idx:
  tr_idx += l
  t_idx += random.choices(l, k=(int(len(l)*0.2)))
print(t_idx)


[[187, 188, 189, 190, 191, 231, 232, 233, 234, 235, 261, 262, 263, 264, 265, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285], [246, 247, 248, 249, 250, 251, 252, 253, 254, 255], [182, 183, 184, 185, 186, 197, 198, 199, 200, 201, 207, 208, 209, 210, 211, 217, 218, 219, 220], [0, 1, 2, 3, 4, 5, 6, 7, 105, 106, 107, 108, 133, 134, 135, 136, 137, 138, 139, 140], [8, 9, 10, 11, 12, 109, 110, 111, 112, 113], [13, 14, 15, 16, 17, 67, 68, 69, 70, 71, 86, 87, 88, 89, 90], [18, 19, 20, 21, 57, 58, 59, 60, 61, 72, 73, 74, 75, 256, 257, 258, 259, 260], [22, 23, 24, 25, 26, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [27, 28, 29, 30, 31, 236, 237, 238, 239, 240], [42, 43, 44, 45, 46], [47, 48, 49, 50, 51, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 145, 146, 147, 148], [52, 53, 54, 55, 56], [62, 63, 64, 65, 66], [76, 77, 78, 79, 80, 91, 92, 93, 94, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 141, 142, 143, 144, 149, 150, 151, 152, 153, 154, 155, 15

In [54]:
train = [x for x in tr_idx if x not in t_idx]
test = [x for x in t_idx]
print(train)
print(test)

[187, 188, 189, 190, 231, 232, 233, 235, 261, 262, 264, 265, 276, 277, 278, 279, 280, 281, 282, 283, 284, 246, 247, 249, 250, 251, 252, 253, 255, 182, 183, 184, 185, 186, 197, 198, 199, 201, 207, 208, 209, 210, 211, 217, 218, 219, 0, 1, 2, 3, 4, 5, 6, 7, 105, 106, 133, 134, 135, 137, 138, 140, 8, 9, 10, 11, 109, 110, 111, 112, 113, 14, 16, 67, 68, 69, 70, 71, 86, 87, 88, 89, 90, 18, 20, 21, 57, 58, 59, 60, 61, 73, 74, 75, 257, 258, 259, 260, 22, 23, 24, 25, 26, 32, 33, 35, 36, 37, 39, 41, 27, 29, 30, 31, 236, 237, 238, 239, 42, 43, 44, 46, 48, 50, 51, 95, 96, 97, 98, 100, 101, 102, 103, 104, 145, 146, 147, 148, 52, 53, 54, 55, 63, 64, 65, 66, 76, 77, 78, 80, 91, 92, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 127, 128, 129, 131, 132, 141, 142, 143, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 171, 172, 173, 81, 82, 84, 85, 174, 175, 176, 177, 178, 179, 180, 181, 213, 214, 215, 216, 203, 204, 205, 206, 222, 223, 241, 242, 243,

In [59]:
X_train = []
Y_train = []

for i in train:
  X_train.append(X[i])
  Y_train.append(Y[i])

X_test = []
Y_test = []
for i in test:
  X_test.append(X[i])
  Y_test.append(Y[i])

In [64]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

print(f'X_train: {X_train}')
# print(f'Y_train: {Y_train}')
print(len(Y_train))

X_train: ['điểm xét tuyển theo phương_thức 2 ưu_tiên xét tuyển có được cộng thêm điểm thưởng khi có các thành_tích khác không ạ ?'
 'điểm xét tuyển theo phương_thức ưu_tiên xét tuyển có được cộng thêm điểm thưởng khi có các thành_tích không ạ ?'
 'điểm xét tuyển theo phương_thức 2 ưu_tiên không tuyển có được cộng thêm điểm thưởng khi có các thành_tích khác xét ạ ?'
 'điểm xét tuyển theo phương_thức deuce ưu_tiên xét tuyển có được cộng thêm điểm thưởng chi có các thành_tích khác không ạ ?'
 'cho em hỏi công_thức tính điểm phương_thức 5 xét tuyển kết_hợp nhiều tiêu_chí ạ ?'
 'cho em hỏi công_thức tính điểm phương_thức 5 xét tuyển kết_hợp nhiều tiêu_chí ạ ?'
 'cho em hỏi công_thức tính điểm phương_thức cinque xét tuyển kết_hợp nhiều tiêu_chí ạ ?'
 'cho em công_thức tính điểm phương_thức 5 xét tuyển kết_hợp nhiều tiêu_chí ạ ?'
 'cho em hỏi là pt5 xét tuyển thì không có đgnl có đủ điều_kiện xét không ạ'
 'cho em hỏi là pt5 xét tuyển thì không có đgnl có đủ điều_kiện xét không ạ'
 'cho em hỏ

In [65]:
# cal TF-IDF
def tf_idf(corpus):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3))
    vec = vectorizer.fit_transform(corpus)
    f_name = vectorizer.get_feature_names_out()
    dense = vec.todense()
    denselist = dense.tolist()
    return denselist, f_name

# X_train = np.append(Q,A)
# X_train = Q
# Y_train = np.append(Q_I,A_I)
# Y_train = I

dl_train, f_train = tf_idf(X_train)
df_train = pd.DataFrame(dl_train, columns=f_train)
# print(f'dl_train: {dl_train}\nf_train: {f_train}\n len: {len(f_train)}')


# dl_test, f_test = tf_idf(X_test)
# df_test = pd.DataFrame(dl_test, columns=f_test)

# print(f'dl_test: {dl_test}\nf_test: {f_test}\n len: {len(f_test)}')

# print(f'f_train - f_test: {np.setdiff1d(f_train, f_test)}\n len: {len(np.setdiff1d(f_train, f_test))}')

max_len = len(f_train)
print(max_len)
# print(type(f_train))

# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

print(X_train.shape[0])

def sen_to_vec(X, vocab, max_len):
      """
      Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
      The output shape should be such that it can be given to `Embedding()` (described in Figure 4).

      Arguments:
      X -- array of sentences (strings), of shape (m, 1)
      vocab -- the list of word (vocab)
      max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.

      Returns:
      X_vec -- occurrence of word in sentence, of shape (m, max_len)
      """

      m = X.shape[0]        # number of training examples

      ### START CODE HERE ###
      # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
      X_vec = np.zeros([m, max_len])

      for i in range(m):                               # loop over training examples

          # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
          word_tokens = [w.lower() for w in str(X[i]).split(' ')]
          no_sw = [w for w in word_tokens if not w in stop_words]
          # stem = [ps.stem(w) for w in no_sw]
          # filtered_sentence = (" ").join(stem)
          # print(stem)

          for j in range(len(no_sw)):
            # print(stem[i])
            # for j in range(len(stem[i])):
            #   print(stem[i][j])
              if no_sw[j] in vocab:
                for k in range(len(vocab)):
                  if no_sw[j] == vocab[k]:
                    # print(stem[i])
                    X_vec[i][k] += 1.0


      ### END CODE HERE ###
      return X_vec

x_train = sen_to_vec(X_train, f_train, max_len)
x_test = sen_to_vec(X_test, f_train, max_len)

print(x_train[0])

# print(f)
# key_word = []
# x_train_mat = []
# x_test_mat = []


2315
237
[0. 0. 0. ... 0. 0. 0.]


In [70]:

# X_train, Y_train = read_csv('E:code/python/chatbot/train.csv')
# print(f'type(Y_train): {type(Y_train)}')
# test, Y_test = read_csv('E:code/python/chatbot/test.csv')
# X_train = tf_idf(lines)
classifier = svm.SVC(kernel='linear', gamma='auto', C=3)

classifier.fit(x_train, Y_train)

# X_test = tf_idf(test)

acc = classifier.score(x_test, Y_test)
print()
print("Test accuracy = ", acc)

# Y_pred = classifier.predict(dl_test)




Test accuracy =  0.9622641509433962


In [71]:
# C = 3
# y_test_oh = np.eye(C)[Y_test.reshape(-1)]
wrong = []
X_test_indices = sen_to_vec(x_test, f_train, max_len)
pred = classifier.predict(X_test_indices)
for i in range(len(x_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        wrong.append(num)
        print(str(i+1) + '.' )
        print('Input: ' + str(X_test[i]))
        print('Expected: '+ str(Y_test[i]))
        print('Prediction: '+ str(num))


1.
Input: khoa kh & kt máy_tính có chương_trình dành cho người vừa đi làm vừa thi_tuyển học hay không ? thí_sinh cần tham_gia đi hay chỉ cần xét tuyển ?
Expected: 1
Prediction: 0
2.
Input: cho em hỏi công_thức phương_thức điểm tính 5 xét tuyển kết_hợp nhiều tiêu_chí ạ ?
Expected: 1
Prediction: 0
3.
Input: cho pica hỏi là pt5 xét tuyển thì không có đgnl có đủ điều_kiện xét không ạ
Expected: 1
Prediction: 0
4.
Input: cho em hỏi công_thức phương_thức điểm tính 5 xét tuyển kết_hợp nhiều tiêu_chí ạ ?
Expected: 1
Prediction: 0
5.
Input: điểm xét tuyển theo phương_thức 2 ưu_tiên xét tuyển có được cộng thêm điểm thưởng khi có các thành_tích khác không ạ ?
Expected: 1
Prediction: 0
6.
Input: cho em hỏi trường xét phương_thức kết_hợp Phoebe là vào ngày mấy ạ
Expected: 2
Prediction: 0
7.
Input: cho em hỏi còn đợt được hồ_sơ cho chương_trình đào_tạo thạc_sĩ phương_thức tuyển thẳng nào không . do em chưa mong đủ hồ_sơ nên đợt nộp ngày 16/6/2023 em không_thể nộp kịp . em chuẩn_bị nộp phản_hồi . xin 

In [16]:
def code_to_str(code):
  # label_dict = {"0": "card delay",
  #                   "1": "card info issue",
  #                   "2": "balance issue"
  #            }
  label_dict = {"1": "Hỏi về phương thức tuyển sinh",
                "2": "Hỏi về thời gian đăng ký",
                "3": "Hỏi về hồ sơ tuyển sinh",
                "4": "so sánh các ngành",
                "5": "chuyển ngành / chương trình học trong tương lai",
                "6": "chọn ngành học",
                "7": "học phí",
                "8": "triển vọng nghề nghiệp",
                "9": "KTX trường",
                "10": "cơ hội du học",
                "11": "kiến thức đầu vào của ngành",
                "12": "hoãn nghĩa vụ quân sự",
                "13": "đào tạo sau đại học",
                "14": "chương trình đào tạo",
                "15": "địa điểm học tập",
                "16": "uy tín đào tạo",
                "17": "thông tin thêm",
                "18": "đăng kí môn học",
                "19": "thời gian trả / công bố kết quả",
                "20": "khối tuyển sinh",
                "21": "vấn đề phần mềm"}
  return label_dict[str(code)]

In [24]:
user = np.array([input('> ')])
user_i = sen_to_vec(user, f_train, max_len)
pred_u = int(classifier.predict(user_i))
print(f'out: {code_to_str(pred_u)}')

> Học phí của cơ điện tử
out: đào tạo sau đại học


In [ ]:
chương trình tiên tiến là gì?
em muốn hỏi về phương thức 2 ưu tiên xét tuyển
em muốn hỏi về phương thức 2
khoa có những ngành học nào
học phí của trường
phương thức xét tuyển 5
các phương thức xét tuyển của trường
Học phí của cơ điện tử

In [22]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.9 MB/s eta

In [30]:
from textattack.augmentation import EasyDataAugmenter

text = "muốn giải bài toán tháp Hà Nội rất khó"

eda_aug = EasyDataAugmenter()

aug = np.array(eda_aug.augment(text))
print(len(aug))
print(aug)

3
['muốn tháp bài toán giải Hà Nội rất khó'
 'muốn giải bài toán tháp Hà Nội rất khó'
 'muốn giải bài tháp Hà Nội rất khó']


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
